# Clustering of bike accident locations

In [1]:
import pandas as pd
import numpy as np
import requests
import folium
import config
from math import radians, cos, sin, asin, sqrt
import os.path
from functools import partial
import pyproj
import math
import csv
import pickle

# DATA VISUALISATION
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import asPolygon
from shapely.geometry import asLineString
from shapely.ops import transform
from shapely.ops import cascaded_union
from shapely.geometry import Point
import geopandas as gpd

# models
from sklearn.cluster import KMeans, AffinityPropagation, DBSCAN
import scipy

In [2]:
#load preprocessed data frame with 2018-2020 data
df = pd.read_csv('../raw_data/df_LOR_2021.csv', index_col=False)

In [ ]:
df

In [ ]:
sns.histplot(df['hour'])

In [ ]:
#plt.scatter(x=df['XGCSWGS84'], y= df['YGCSWGS84'])

In [ ]:
#sns.histplot(x = df['LINREFX'])

In [ ]:
#sns.histplot(x = df['LINREFY'])

In [ ]:
# don't need to scale! instantiating scaler
# scaler = StandardScaler()

In [ ]:
#scaler.fit(df[['XGCSWGS84']])

In [ ]:
# df['XGCSWGS84'] = scaler.transform(df[['XGCSWGS84']])

In [ ]:
#scaler2 = StandardScaler()

In [ ]:
#scaler2.fit(df[['YGCSWGS84']])

In [ ]:
#df['YGCSWGS84'] = scaler2.transform(df[['YGCSWGS84']])

In [ ]:
#plt.scatter(x=df['XGCSWGS84'], y= df['YGCSWGS84'])

## K-Means

In [ ]:
%%time
# elbow method for number of clusters

X = df[["LINREFX","LINREFY"]]
max_k = 100

## iterations
distortions = [] 
for i in range(1, max_k+1):
    if len(X) >= i:
        model= KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
        model.fit(X)
        distortions.append(model.inertia_)

## best k: the lowest derivative
k = [i*100 for i in np.diff(distortions,2)].index(min([i*100 for i in np.diff(distortions,2)]))

## plot
fig, ax = plt.subplots()
ax.plot(range(1, len(distortions)+1), distortions)
ax.axvline(k, ls='--', color="red", label="k = "+str(k))
ax.set(title='Elbow Method', xlabel='Number of clusters', 
       ylabel="Distortion")
ax.legend()
ax.grid(True)
plt.show()

In [ ]:
%%time

k = 2000
model = KMeans(n_clusters=k, init='k-means++')
X = df[["LINREFX","LINREFY"]]

## clustering
df_X = X.copy()
df_X["cluster"] = model.fit_predict(X)

## find centroids (the datapoints closest to the center)
closest, distances = scipy.cluster.vq.vq(model.cluster_centers_, 
                     df_X.drop("cluster", axis=1).values)
df_X["centroids"] = 0
for i in closest:
    df_X["centroids"].iloc[i] = 1
    
## add clustering info to the original dataset
df[["cluster_km","centroids_km"]] = df_X[["cluster","centroids"]]
df.sample(5)

In [ ]:
## plot
fig, ax = plt.subplots()
sns.scatterplot(x="LINREFX", y="LINREFY", data=df, 
                palette=sns.color_palette("bright",k),
                hue='cluster', size="centroids", size_order=[1,0],
                legend=False, ax=ax).set_title('Clustering(k='+str(k)+')')
                                                 
th_centroids = model.cluster_centers_
ax.scatter(th_centroids[:,0], th_centroids[:,1], s=50, c='black', 
           marker="x")

In [ ]:
# df that contains only centroids for mappinig

df_kmeans_cluster = df[df['centroids'] == 1]

df_kmeans_cluster.head()

## Affinity propagation

In [ ]:
%%time

model = AffinityPropagation()
X = df[["LINREFX","LINREFY"]]

## clustering
df_X = X.copy()
df_X["cluster"] = model.fit_predict(X)

## find real centroids
closest, distances = scipy.cluster.vq.vq(model.cluster_centers_, 
                     df_X.drop("cluster", axis=1).values)
df_X["centroids"] = 0
for i in closest:
    df_X["centroids"].iloc[i] = 1
    
## add clustering info to the original dataset
df[["cluster_ap","centroids_ap"]] = df_X[["cluster","centroids"]]
df.sample(5)

In [ ]:
#model = AffinityPropagation()

In [ ]:
#model, dtf_X = fit_ml_cluster(X=df[["XGCSWGS84","YGCSWGS84"]], model=model, figsize=(10,5))

In [ ]:
'''
## plot
fig, ax = plt.subplots()
sns.scatterplot(x="LINREFX", y="LINREFY", data=df, 
                palette=sns.color_palette("bright",k),
                hue='cluster', size="centroids", size_order=[1,0],
                legend=False, ax=ax).set_title('Clustering(k='+str(k)+')')
                                                 
th_centroids = model.cluster_centers_
ax.scatter(th_centroids[:,0], th_centroids[:,1], s=50, c='black', 
           marker="x")
''';

## DBSCAN

In [ ]:
# find clusters using DB_SCAN (density based)




In [3]:
# create_radian_columns(df):

df['rad_lng'] = np.radians(df['XGCSWGS84'].values)
df['rad_lat'] = np.radians(df['YGCSWGS84'].values)

In [4]:
# find clusters with DBSCAN:

eps_in_meters = 25 # hotspot area
num_samples = 5 # accidents in cluster
earth_perimeter = 40070000.0  # In meters
eps_in_radians = eps_in_meters / earth_perimeter * (2 * math.pi)
db_scan = DBSCAN(eps=eps_in_radians, min_samples=num_samples, metric='haversine')
df['Cluster'] = db_scan.fit_predict(df[['rad_lat', 'rad_lng']])

In [5]:
def buffer_in_meters(lng, lat, radius):
    proj_meters = pyproj.Proj(init='epsg:3857')
    proj_latlng = pyproj.Proj(init='epsg:4326')
    
    project_to_meters = partial(pyproj.transform, proj_latlng, proj_meters)
    project_to_latlng = partial(pyproj.transform, proj_meters, proj_latlng)
    
    pt_latlng = Point(lng, lat)
    pt_meters = transform(project_to_meters, pt_latlng)
    
    buffer_meters = pt_meters.buffer(radius)
    
    buffer_latlng = transform(project_to_latlng, buffer_meters)
    
    return buffer_latlng
    

In [ ]:
#def create_point(row):
#    return Point(row['Longitude'], row['Latitude'])

In [ ]:
#def write_line_string(file_name, hull):
#    with open("data/out/{0}.csv".format(file_name), "w") as file:
#        file.write('\"line\"\n')
#        text = hull.wkt
#        file.write('\"' + text + '\"\n')

In [6]:
# Group the observations by cluster identifier

#def generate_blob_clusters(df, eps_in_meters=50):

groups = df.groupby('Cluster')
clusters = list()
blobs = list()
counts = list()
    
for cluster_id, points in groups:
    if cluster_id >= 0:
        buffer_radius = eps_in_meters #* 0.6
        buffers = [buffer_in_meters(lon, lat, buffer_radius)for lon, lat in zip(points['XGCSWGS84'], points['YGCSWGS84'])]
        blob = cascaded_union(buffers)
        blobs.append(blob)
        clusters.append(cluster_id)
        counts.append(len(points))
            
# Create the GeoDataFrame from the cluster numbers and blobs
data = {'cluster': clusters, 'polygon': blobs, 'count': counts}
cluster_gdf = gpd.GeoDataFrame(pd.DataFrame(data), geometry='polygon')
cluster_gdf.crs = {'init': 'epsg:4326'}

/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packag

/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packag

/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packag

/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packag

/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packag

...

/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packag

/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packag

/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packag

/tmp/ipykernel_3799/3630490427.py:14: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  blob = cascaded_union(buffers)
/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-cha

/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/supersakul/.pyenv/versions/3.8.12/envs/berlin-bike-theft-forecasting/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_3799/3630490427.py:14: ShapelyDeprecationWarning: The 'cascaded_union()' function is 

In [7]:
cluster_gdf['count'].value_counts()

5     146
6      86
7      52
8      48
9      33
10     21
11     14
12     10
13      9
15      7
14      7
17      4
19      3
20      3
18      3
22      3
16      2
4       2
25      1
30      1
Name: count, dtype: int64

In [8]:
cluster_gdf = cluster_gdf[cluster_gdf['count'] > 4]

In [22]:
cluster_gdf.to_csv('../raw_data/B_Accident_data/cluster_gdf.csv')

In [ ]:
c

In [19]:
cluster_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 453 entries, 0 to 454
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   cluster  453 non-null    int64   
 1   polygon  453 non-null    geometry
 2   count    453 non-null    int64   
dtypes: geometry(1), int64(2)
memory usage: 30.3 KB


In [21]:
cluster_gdf.polygon

0      POLYGON ((13.43232 52.53513, 13.43230 52.53512...
1      POLYGON ((13.47078 52.51403, 13.47077 52.51402...
2      POLYGON ((13.44868 52.50478, 13.44867 52.50477...
3      POLYGON ((13.45395 52.51870, 13.45394 52.51869...
4      POLYGON ((13.43640 52.51737, 13.43637 52.51737...
                             ...                        
450    POLYGON ((13.31226 52.50031, 13.31226 52.50030...
451    POLYGON ((13.40174 52.52938, 13.40173 52.52937...
452    POLYGON ((13.38643 52.55848, 13.38642 52.55847...
453    POLYGON ((13.42741 52.48098, 13.42739 52.48098...
454    POLYGON ((13.35231 52.50515, 13.35229 52.50514...
Name: polygon, Length: 453, dtype: geometry

In [ ]:
cluster_gdf.to_pickle('../raw_data/B_Accident_data/cluster_gdf.pickle')


In [ ]:
##density = (df
##    .reset_index()
##    .rename(columns={'Id': 'Count'})
##    .groupby('Postcode cijfers')
##    .count()
##)[['Count']].reset_index()
#
#
#
#choropleth = folium.Choropleth('../Resources/PC4_BUURTEN.json',
#                  name='Accident count',
#                  data=cluster_gdf,
#                  key_on='properties.Postcode4',
#                  columns=['Count'],
#                  fill_color='YlGn',
#                  fill_opacity=0.7,
#                  line_opacity=0.2,
#                  legend_name='Customer Count',
#).add_to(m)
#
#choropleth.geojson.add_child(
#    folium.features.GeoJsonTooltip(['Postcode4'])
#)

In [ ]:
#plotting the hotspots in folium
m = folium.Map(location=[52.52000, 13.4050],tiles='openstreetmap', zoom_start=12)
folium.features.GeoJson(cluster_gdf,
                        style_function= lambda x: {'fillColor': 'red', 'color': 'red'},
                        tooltip=folium.GeoJsonTooltip(fields= ['count'],aliases=["Accident count: "],labels=True)).add_to(m)

In [ ]:
# Filter out the noise points and retain only the clusters
#cluster_points = df.loc[df['Cluster'] > -1, ['Cluster', 'XGCSWGS84', 'YGCSWGS84']]

## Creating custom hotspot maps

### Hotspots with >20 accidents

In [ ]:
# Hotspot map only > 20 accidents
hotspot20 = cluster_gdf.copy()

In [ ]:
hotspot20 = hotspot20[hotspot20['count'] >= 15]

In [ ]:
hotspot20

In [ ]:
folium.PolyLine(locations=completed, color='green', weight=7, opacity=0.7, tooltip='completed').add_to(m_hotspot20)
folium.PolyLine(locations=under_construction, color='yellow', weight=7, opacity=0.7, tooltip='under construction').add_to(m_hotspot20)
folium.PolyLine(locations=in_planning, color='orange', weight=7, opacity=0.7, tooltip='in planning').add_to(m_hotspot20)
folium.PolyLine(locations=intended, color='red', weight=7, opacity=0.7, tooltip='intended').add_to(m_hotspot20)

In [ ]:
m_hotspot20 = folium.Map(location=[52.52000, 13.4050],tiles='openstreetmap', zoom_start=12)
folium.features.GeoJson(hotspot20,
                        style_function= lambda x: {'fillColor': 'red', 'color': 'red'},
                        tooltip=folium.GeoJsonTooltip(fields= ['count'],aliases=["Accident count: "],labels=True)).add_to(m_hotspot20)

In [ ]:
m_hotspot20

### Hotspots during rush hours

In [ ]:
df['rad_lng'] = np.radians(df['XGCSWGS84'].values)
df['rad_lat'] = np.radians(df['YGCSWGS84'].values)

In [ ]:
df.head()

In [ ]:
df_rushhour1 = df[df['hour'].between(7, 9, inclusive = 'both')]

In [ ]:
df_rushhour2 = df[df['hour'].between(17, 19, inclusive = 'both')]

In [ ]:
df_rushhour = pd.concat([df_rushhour1, df_rushhour2])

In [ ]:
df_rushhour

In [ ]:
# Percentage of accidents within rush hours:
a_in_rh = 100/15303*6306
a_in_rh

In [ ]:
# find clusters with DBSCAN:

eps_in_meters = 25 # hotspot area
num_samples = 5 # accidents in cluster
earth_perimeter = 40070000.0  # In meters
eps_in_radians = eps_in_meters / earth_perimeter * (2 * math.pi)
db_scan = DBSCAN(eps=eps_in_radians, min_samples=num_samples, metric='haversine')
df_rushhour['Cluster'] = db_scan.fit_predict(df_rushhour[['rad_lat', 'rad_lng']])

In [ ]:
# Group the observations by cluster identifier

#def generate_blob_clusters(df, eps_in_meters=50):

groups_rh = df_rushhour.groupby('Cluster')
clusters_rh = list()
blobs_rh = list()
counts_rh = list()
    
for cluster_id, points in groups_rh:
    if cluster_id >= 0:
        buffer_radius = eps_in_meters #* 0.6
        buffers = [buffer_in_meters(lon, lat, buffer_radius)for lon, lat in zip(points['XGCSWGS84'], points['YGCSWGS84'])]
        blob = cascaded_union(buffers)
        blobs_rh.append(blob)
        clusters_rh.append(cluster_id)
        counts_rh.append(len(points))
            
# Create the GeoDataFrame from the cluster numbers and blobs
data_rh = {'cluster': clusters_rh, 'polygon': blobs_rh, 'count': counts_rh}
cluster_rh_gdf = gpd.GeoDataFrame(pd.DataFrame(data_rh), geometry='polygon')
cluster_rh_gdf.crs = {'init': 'epsg:4326'}

In [ ]:
cluster_rh_gdf

In [ ]:
#plotting the hotspots in folium
m_rh = folium.Map(location=[52.52000, 13.4050],tiles='openstreetmap', zoom_start=12)
folium.features.GeoJson(cluster_rh_gdf,
                        style_function= lambda x: {'fillColor': 'red', 'color': 'red'},
                        tooltip=folium.GeoJsonTooltip(fields= ['count'],aliases=["Accident count: "],labels=True)).add_to(m_rh)

In [ ]:
m_rh

In [ ]:
cluster_rh_gdf.to_pickle('../raw_data/B_Accident_data/cluster_gdf.pickle')

### Fatal accidents

In [ ]:
#creating filtered df with only fatal accidents
df_fa = df[df['category'] == 1]

In [ ]:
#creating tuple only containing coordinates
fa = tuple(np.array([df_fa.YGCSWGS84, df_fa.XGCSWGS84]).T.tolist())

In [ ]:
#Plotting

m_fa = folium.Map(location=[52.52000, 13.4050],tiles='openstreetmap', zoom_start=12)
for each in fa:
    folium.CircleMarker(each, radius=2, color='red').add_to(m_fa)

In [ ]:
m_fa

In [ ]:
df_test

## Greenlane data integration

In [ ]:
final_df = pd.read_pickle('../raw_data/B_Accident_data/greenlane_df.pickle')

In [ ]:
final_df = final_df[['id', 'status', 'coordinates']]

In [ ]:
final_df.head()

In [ ]:
completed = final_df.coordinates[final_df.status == 'completed']
under_construction = final_df.coordinates[final_df.status == 'under construction']
in_planning = final_df.coordinates[final_df.status == 'in planning']
intended = final_df.coordinates[final_df.status == 'intended']

In [ ]:
folium.PolyLine(locations=completed, color='green', weight=7, opacity=0.7, tooltip='completed').add_to(m)
folium.PolyLine(locations=under_construction, color='yellow', weight=7, opacity=0.7, tooltip='under construction').add_to(m)
folium.PolyLine(locations=in_planning, color='orange', weight=7, opacity=0.7, tooltip='in planning').add_to(m)
folium.PolyLine(locations=intended, color='red', weight=7, opacity=0.7, tooltip='intended').add_to(m)

In [ ]:
m

## SOMs

In [ ]:
# find clusters using SOMs (self organizing maps)

X = df[["XGCSWGS84","YGCSWGS84"]]
map_shape = (4,4)


model = minisom.MiniSom(x=map_shape[0], y=map_shape[1], 
                        input_len=X.shape[1])
model.train_batch(X, num_iteration=100, verbose=1)

## build output dataframe
dtf_X = X.copy()
dtf_X["cluster"] = np.ravel_multi_index(np.array(
      [model.winner(x) for x in X_preprocessed]).T, dims=map_shape)## find real centroids
cluster_centers = np.array([vec for center in model.get_weights() 
                            for vec in center])closest, distances = scipy.cluster.vq.vq(model.cluster_centers_, 
                                         X_preprocessed)
dtf_X["centroids"] = 0
for i in closest:
    dtf_X["centroids"].iloc[i] = 1## add clustering info to the original dataset
dtf[["cluster","centroids"]] = dtf_X[["cluster","centroids"]]## plot
k = dtf["cluster"].nunique()fig, ax = plt.subplots()
sns.scatterplot(x="Latitude", y="Longitude", data=dtf, 
                palette=sns.color_palette("bright",k),
                hue='cluster', size="centroids", size_order=[1,0],
                legend="brief", ax=ax).set_title('Clustering 
                (k='+str(k)+')')th_centroids = scaler.inverse_transform(cluster_centers)
ax.scatter(th_centroids[:,0], th_centroids[:,1], s=50, c='black', 
           marker="x")